<a href="https://colab.research.google.com/github/syuan0525/project-team/blob/main/modelgenerator1118.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import datasets, models, layers, utils, activations, losses, optimizers, metrics
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, BatchNormalization, Dropout

base_model = MobileNetV2(
    include_top=False, weights='imagenet', input_shape=(224, 224, 3)
)
x = base_model.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.25)(x)
predictions = layers.Dense(20, activation='softmax')(x)

# Freeze layers 凍結層 把遷移來的卷積的參數固定
for l in base_model.layers:
  l.trainable = False

# 把卷積和後面的分類層合在一起
model = models.Model(base_model.input, predictions)
model.summary()
# base_model.summary()

9420800/9406464 [==============================] - 0s 0us/step
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 112, 112, 32  128         ['Conv1[0][0]']                  
                               

In [3]:
from tensorflow.keras.losses import SparseCategoricalCrossentropy
model.compile(
        loss=SparseCategoricalCrossentropy(),
        optimizer="adam",
        metrics=["acc"]
)

In [4]:
import glob
import os
os.chdir("/content/drive/MyDrive/Tebame_第四組_機車辨識")

In [5]:
s2 = glob.glob("./嚴選資料集/GOGORO_S2_裁/*.jpg") 

gp = glob.glob("./嚴選資料集/KYMCO_GP_125_裁/*") 

krv = glob.glob("./嚴選資料集/KYMCO_KRV_裁/*")

fiddle = glob.glob("./嚴選資料集/SYM_FIDDLE_125_裁/*") 

sr = glob.glob("./嚴選資料集/SYM_JET_SR_125_裁/*")

vivo = glob.glob("./嚴選資料集/SYM_活力_125_裁/*")

famous = glob.glob("./嚴選資料集/KYMCO_新名流125_裁/*")

drg = glob.glob("./嚴選資料集/SYM_DRG_裁/*") 

duke = glob.glob("./嚴選資料集/SYM_新迪爵_125 _裁/*")

limi = glob.glob("./嚴選資料集/YAMAHA_LIMI_125_已裁減/*")

gryphus6 = glob.glob("./嚴選資料集/YAMAHA_六代勁戰_125_裁/*")

force = glob.glob("./嚴選資料集/YAMAHA_FORCE_裁/*")

viva = glob.glob("./嚴選資料集/GOGORO_VIVA_MIX_裁/*")

s3 = glob.glob("./嚴選資料集/GOGORO_S3_裁/*")

like = glob.glob("./嚴選資料集/KYMCO_LIKE_125_裁/*")

many = glob.glob("./嚴選資料集/KYMCO_MANY_125_裁/*")

bws = glob.glob("./嚴選資料集/YAMAHA_BWS_125_裁/*")

sl = glob.glob("./嚴選資料集/SYM_JET_SL_125_裁/*")

axiz = glob.glob("./嚴選資料集/YAMAHA_勁豪_125_裁/*")

jog = glob.glob("./嚴選資料集/YAMAHA_JOG_125_裁/*")

In [6]:
print("s2 共:", len(s2))
print("gp 共:", len(gp))
print("krv 共:", len(krv))
print("fiddle 共:", len(fiddle))
print("sr 共:", len(sr))
print("vivo 共:", len(vivo))
print("famous 共:", len(famous))
print("drg 共:", len(drg))
print("duke 共:", len(duke))
print("limi 共:", len(limi))
print("gryphus6 共:", len(gryphus6))
print("force 共:", len(force))
print("viva 共:", len(viva))
print("s3 共:", len(s3))
print("like 共:", len(like))
print("many 共:", len(many))
print("bws 共:", len(bws))
print("sl 共:", len(sl))
print("axiz 共:", len(axiz))
print("jog 共:", len(jog))

s2 共: 327
gp 共: 574
krv 共: 336
fiddle 共: 790
sr 共: 211
vivo 共: 477
famous 共: 618
drg 共: 401
duke 共: 545
limi 共: 315
gryphus6 共: 788
force 共: 358
viva 共: 182
s3 共: 315
like 共: 276
many 共: 354
bws 共: 142
sl 共: 98
axiz 共: 511
jog 共: 212


In [9]:
import tensorflow as tf
import cv2
from skimage import io
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import imgaug.augmenters as iaa
import imgaug as ia
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input 

In [12]:
class DataGenerator(tf.keras.utils.Sequence):
    def __init__(self, img_paths, batch_size, img_size, shuffle=True, aug=False):
        self.img_paths = img_paths
        self.batch_size = batch_size
        self.img_size = img_size
        self.shuffle = shuffle
        self.aug = aug

        self.indexes = np.arange(len(self.img_paths)) # [0, 1, 2, 3, 4, 5]
        self.on_epoch_end()

    def __len__(self):
        return int(np.ceil(len(self.img_paths) / self.batch_size)) # batches per epoch

    def __getitem__(self, index):
        # Generate indexes of the batch
        idxs = self.indexes[index * self.batch_size:(index + 1) * self.batch_size]
        # Find list of IDs
        batch_img_paths = [self.img_paths[i] for i in idxs]

        # Generates data containing batch_size samples
        x = np.empty((len(batch_img_paths), self.img_size, self.img_size, 3), dtype=np.float32)
        y = np.empty((len(batch_img_paths), 1), dtype=np.float32)
        # y = np.array(df["ans"])

        for i, img_path in enumerate(batch_img_paths):
            # read img
            # img = cv2.imread(img_path)
            # img = cv2.resize(img, (self.img_size, self.img_size))
            # img = img / 255.

            img = Image.open(img_path).resize((self.img_size, self.img_size)).convert("RGB")
            img = np.array(img)
            img = preprocess_input(img)

            if self.aug:
              seq = iaa.Sequential([
                      iaa.Fliplr(0.5), # 50% horizontal flip
                      iaa.Flipud(0), # 50% vertical flip
                      iaa.Affine(
                      rotate=(-45, 45), # random rotate -45 ~ +45 degree
                      shear=(-16, 16), # random shear -16 ~ +16 degree
                      scale={"x": (0.8, 1.2), "y": (0.8, 1.2)}, # scale x, y: 80%~120%,
                      mode='constant', # ia.ALL, 'constant','edge', 'symmetric', 'reflect', 'wrap'
                      cval=0,
                        ),
                      ])
              img_aug = seq.augment_image(img)
              img = img_aug # Augmentation

            # read class label
            cls = img_path.split('/')[-2]
            if "S2" in cls:
                cls = 0
            elif "GP" in cls:
                cls = 1
            elif "KRV" in cls:
                cls = 2
            elif "FIDDLE" in cls:
                cls = 3
            elif "SR" in cls:
                cls = 4
            elif "活力" in cls:
                cls = 5
            elif "新名流" in cls:
                cls = 6
            elif "DRG" in cls:
                cls = 7
            elif "新迪爵" in cls:
                cls = 8
            elif "LIMI" in cls:
                cls = 9
            elif "六代" in cls:
                cls = 10
            elif "FORCE" in cls:
                cls = 11
            elif "VIVA" in cls:
                cls = 12
            elif "S3" in cls:
                cls = 13
            elif "LIKE" in cls:
                cls = 14
            elif "MANY" in cls:
                cls = 15
            elif "BWS" in cls:
                cls = 16
            elif "SL" in cls:
                cls = 17
            elif "勁豪" in cls:
                cls = 18
            elif "JOG" in cls:
                cls = 10
            x[i] = img
            y[i] = cls
            
        return x, y

    def on_epoch_end(self):
        # Updates indexes after each epoch
        if self.shuffle:
            np.random.shuffle(self.indexes)

In [11]:
img_paths = s2 + gp + krv + fiddle + sr + vivo + famous + drg + duke + limi + gryphus6
len(img_paths)

5382

In [13]:
from sklearn.model_selection import train_test_split
train_img_paths, val_img_paths = train_test_split(img_paths, test_size=0.1)
len(train_img_paths), len(val_img_paths)

(4843, 539)

In [14]:
train_gen = DataGenerator(train_img_paths, 32, 224, shuffle=True, aug=True)
val_gen = DataGenerator(val_img_paths, 32, 224, shuffle=False, aug=False)

In [15]:
train_gen.indexes

array([1301,  876, 3326, ..., 1215, 1262,  444])

In [ ]:
len(train_gen)

114

In [ ]:
imgs, labels = train_gen[2]

In [ ]:
imgs.shape, labels.shape

((32, 224, 224, 3), (32, 1))

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
callbacks = [
             EarlyStopping(patience=10, restore_best_weights=True),
             ModelCheckpoint("/content/1109.h5", save_best_only=True)
]

model.fit(
    train_gen,
    steps_per_epoch=10,
    epochs=300,
    validation_data=val_gen,
    validation_steps=2, 
    callbacks=callbacks
                    )

Epoch 1/300
10/10 [==============================] - 144s 11s/step - loss: 2.5644 - acc: 0.0906 - val_loss: 2.5784 - val_acc: 0.0156
Epoch 2/300
10/10 [==============================] - 90s 9s/step - loss: 2.2258 - acc: 0.0969 - val_loss: 2.1975 - val_acc: 0.0781
Epoch 3/300
10/10 [==============================] - 84s 8s/step - loss: 2.0104 - acc: 0.1187 - val_loss: 1.9143 - val_acc: 0.1094
Epoch 4/300
 6/10 [=================>............] - ETA: 29s - loss: 1.8889 - acc: 0.0833